<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_billionaires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Billionaires   



##Problem Statement     

<h4><b>CST3512 Spring 2022 Final Exam Exercise</b></h4>    


<h3><u>Billionaires: 2014 and 2022</u></h3>    

<h4>Goal:</h4>    
Produce two sets of charts, each set has two plots:    

1. bar charts of the net worth of the ten wealthiest billionaires for 2014 and 2022, respectively        

2. pie charts with the count of the number of billionaires in each of the ten countries with the greatest number of billionaires in 2014 and 2022, respectively      


<h4>STEP 1</h4>    

Read the `.csv` file of global billionaire data for 2014    
You may use the following statement to read the file:    
`!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/forbes_billionaires_2014.csv -o 'forbes_billionaires_2014.csv'` 


Read the .csv file of global billionaire data for 2022    
You may use the following statement to read the file:    
`!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/forbes_billionaires_2022.csv  -o 'forbes_billionaires_2022.csv'`    


<h4>STEP 2</h4>    

Display plots* of two bar charts each showing:     
1. the total wealth (in USD, millions) for the top-ten billionaires in 2014           
2. the total wealth (in USD, millions) of the top-ten billionaires in 2022        


<h4>STEP 3</h4>    

Display plots* of two pie charts, each showing:    
1. the count of billionaires by country for the ten countries with the most billionaires in 2014      
1. the count of billionaires by country for the ten countries with the most billionaires in 2022     



*note: if you encounter trouble producing plots, a table with the information which would be used in the plot can count toward partial credit.*   



##H. Housekeeping    

Import libraries, initialize standing data, read data sources.    


Import the required libraries.  

In [1]:
import pandas as pd                        # the usual suspect 

import numpy as np                         # not needed (yet)

import plotly.express as px                # plotly express plots

from plotly.subplots import make_subplots  # for side-by-side plots

import plotly.graph_objects as go          # plotly plots 

from plotly.offline import plot            # plotly plots


No standing data to initialize.    


Read data sources:    

* **`forbes_Billionaires_2014.csv`**    
* **`forbes_Billionaires_2022.csv`**   

*note: the source data files have different formats, including different column names and different scales for the net worth of individuals (millions vs. billions)*

In [2]:
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/forbes_billionaires_2014.csv -o 'forbes_billionaires_2014.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  593k  100  593k    0     0  2760k      0 --:--:-- --:--:-- --:--:-- 2760k


In [3]:
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/forbes_billionaires_2022.csv  -o 'forbes_billionaires_2022.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1949k  100 1949k    0     0  3777k      0 --:--:-- --:--:-- --:--:-- 3777k




---



##1. Process the 2014 Data    


###1.a ETL    


In [2]:
# read the `csv` file of 2014 billionaire data retaining only columns 1, 10, 15
b2014_df = pd.read_csv('forbes_billionaires_2014.csv', usecols=[0, 10, 15])

In [3]:
# create a `year` column with the integer value 2014 
b2014_df['year'] = 2014

In [4]:
# use `.iloc()` to peek at a row in the dataframe -- note the column names 
b2014_df.iloc[0]

name                           Bill Gates
location.citizenship        United States
wealth.worth in billions             18.5
year                                 2014
Name: 0, dtype: object

In [5]:
# review the column names using `.columns`   
b2014_df.columns

Index(['name', 'location.citizenship', 'wealth.worth in billions', 'year'], dtype='object')

In [6]:
# use `.rename()` and dictionary arguments to rename columns (inplace)
b2014_df.rename(columns={'name' : 'personName'}, inplace=True)
b2014_df.rename(columns={'location.citizenship' : 'countryOfCitizenship'}, inplace=True)
b2014_df.rename(columns={'wealth.worth in billions' : 'finalWorth'}, inplace=True)


In [7]:
# transform net worth (`finalWorth`) to millions for comparability with 2022 data 
b2014_df['finalWorth']  = b2014_df['finalWorth'] * 1000

---

###1.b The Top Ten Billionaires by Net Worth in 2014     




In [8]:
# sort the dataframe in descending order (inplace) by net worth (`finalWorth)    
b2014_df.sort_values(by=['finalWorth', 'countryOfCitizenship', 'year', 'personName' ], ascending=False, inplace=True)

In [9]:
# create a new dataframe `t2014_df` with a row for each top-ten billionaire in 2014   
t2014_df = b2014_df[:10][['personName', 'finalWorth']]

In [10]:
# t2014_df.columns = ['Name', 'Worth_2014']   # deprecated this hard-coded assignment
# use `.rename()` and dictionary arguments to rename columns (inplace)
t2014_df.rename(columns={'personName' : 'Name'}, inplace=True)
t2014_df.rename(columns={'finalWorth' : 'Worth_2014'}, inplace=True)

In [11]:
t2014_df

,Name,Worth_2014
2,Bill Gates,76000.0
5,Carlos Slim Helu,72000.0
8,Amancio Ortega,64000.0
1,Bill Gates,58700.0
11,Warren Buffett,58200.0
14,Larry Ellison,48000.0
18,David Koch,40000.0
17,Charles Koch,40000.0
23,Sheldon Adelson,38000.0
26,Christy Walton,36700.0




---



###1.c Top Ten Countries by Count of Billionaires in 2014     


Count billionaires by country using `.groupby()`    

In [12]:
# create a country count dataframe `c2014_df` with the top-ten countries by count of billionaires in 2014 
c2014_df = b2014_df[['countryOfCitizenship','finalWorth']].groupby(['countryOfCitizenship'])['finalWorth'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

In [13]:
# c2014_df.columns = ['Country', 'Billionaires_in_2014']     # deprecated this hard-coded assignment
# use `.rename()` and dictionary arguments to rename columns (inplace)
c2014_df.rename(columns={'countryOfCitizenship' : 'Country'}, inplace=True)
c2014_df.rename(columns={'count' : 'Billionaires_in_2014'}, inplace=True)

In [14]:
c2014_df

,Country,Billionaires_in_2014
70,United States,903
21,Germany,160
11,China,153
52,Russia,119
30,Japan,96
8,Brazil,81
24,Hong Kong,77
19,France,72
69,United Kingdom,65
25,India,63




---



##2. Process the 2022 Data    


###2.a ETL    


In [15]:
# read the `csv` file of 2022 billionaire data retaining only columns 1, 3, 11
b2022_df = pd.read_csv('forbes_billionaires_2022.csv', usecols=[1,3,11])

In [16]:
# create a `year` column with the integer value 2014 
b2022_df['year'] = 2022

In [17]:
# use `.iloc()` to peek at a row in the dataframe -- note the column names 
b2022_df.iloc[0]

personName                  Elon Musk
finalWorth                   219000.0
countryOfCitizenship    United States
year                             2022
Name: 0, dtype: object

In [18]:
# review the column names using `.columns`   
b2022_df.columns

Index(['personName', 'finalWorth', 'countryOfCitizenship', 'year'], dtype='object')

In [19]:
# no need to use `.rename()` and dictionary arguments to rename columns (inplace)
# column names from this 2022 data were used to rename columns in 2014 data


---

###2.b The Top Ten Billionaires by Net Worth in 2022     




In [20]:
# sort the dataframe in descending order (inplace) by net worth (`finalWorth)    
b2022_df.sort_values(by=['finalWorth', 'countryOfCitizenship', 'year', 'personName' ], ascending=False, inplace=True)

In [21]:
# create a new dataframe `t2014_df` with a row for each top-ten billionaire in 2014   
t2022_df = b2022_df[:10][['personName', 'finalWorth']]

In [22]:
# t2022_df.columns = ['Name', 'Worth_2022']   # deprecated this hard-coded assignment
# use `.rename()` and dictionary arguments to rename columns (inplace)
t2022_df.rename(columns={'personName' : 'Name'}, inplace=True)
t2022_df.rename(columns={'finalWorth' : 'Worth_2022'}, inplace=True)

In [23]:
t2022_df

,Name,Worth_2022
0,Elon Musk,219000.0
1,Jeff Bezos,171000.0
2,Bernard Arnault & family,158000.0
3,Bill Gates,129000.0
4,Warren Buffett,118000.0
5,Larry Page,111000.0
6,Sergey Brin,107000.0
7,Larry Ellison,106000.0
8,Steve Ballmer,91400.0
9,Mukesh Ambani,90700.0




---



###2.c Top Ten Countries by Count of Billionaires in 2022     


Count billionaires by country using `.groupby()`    

In [24]:
# create a country count dataframe `c2022_df` with the top-ten countries by count of billionaires in 2022 
c2022_df = b2022_df[['countryOfCitizenship','finalWorth']].groupby(['countryOfCitizenship'])['finalWorth'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

In [25]:
# c2022_df.columns = ['Country', 'Billionaires_in_2022']     # deprecated this hard-coded assignment
# use `.rename()` and dictionary arguments to rename columns (inplace)
c2022_df.rename(columns={'countryOfCitizenship' : 'Country'}, inplace=True)
c2022_df.rename(columns={'count' : 'Billionaires_in_2022'}, inplace=True)

In [26]:
c2022_df

,Country,Billionaires_in_2022
70,United States,735
11,China,539
28,India,166
22,Germany,134
54,Russia,83
25,Hong Kong,67
9,Canada,64
7,Brazil,62
32,Italy,52
63,Taiwan,51



---


##3. Plots    


###3.a <u>Bar Charts of 10 Wealthiest Billionaires [2014, 2022]</u>    


In [27]:
fig = px.histogram(t2014_df, x='Name', y='Worth_2014').update_xaxes(categoryorder='total ascending')

fig.update_layout(title_text='Top Ten Wealthiest Billionaires in 2014', title_x=0.5)

fig.show()

In [28]:
fig = px.histogram(t2022_df, x='Name', y='Worth_2022').update_xaxes(categoryorder='total ascending')

fig.update_layout(title_text='Top Ten Wealthiest Billionaires in 2022', title_x=0.5)

fig.show()

###3.b <u>Top Ten Countries with the Most Billionaires [2014, 2022]</u>    


In [29]:
fig = px.pie(c2014_df, values ="Billionaires_in_2014", names='Country')

fig.update_layout(title_text='Top Ten Countries by Count of Billionaires in 2014', title_x=0.5)

fig.update_traces(hoverinfo='label+value', textinfo='value')

fig.show()

In [30]:
fig = px.pie(c2022_df, values ="Billionaires_in_2022", names='Country')

fig.update_layout(title_text='Top Ten Countries by Count of Billionaires in 2022', title_x=0.5)

fig.update_traces(hoverinfo='label+value', textinfo='value')

fig.show()



---




<h1><center>STRAY SNIPPETS FOLLOW</center></h1>

---



In [ ]:
# concatenating dataframes
frames = [b2014_df, b2022_df]
billionaires_df = pd.concat(frames)

In [ ]:
# groupby with reset of index
b_df = billionaires_df.groupby(["countryOfCitizenship","year"]).finalWorth.sum().reset_index()

In [ ]:
# finding nlargest 
p2014_df = b2014_df.nlargest(20, 'finalWorth')
p2022_df = b2022_df.nlargest(20, 'finalWorth')

In [ ]:
# selecting billionaires with net worth over 100 billion 
billionaires_df.loc[billionaires_df['finalWorth'] > 100000]


---
